In [598]:
import pandas as pd
import numpy as np

import utilities
import json

from py2neo import Graph

from sklearn.linear_model import LinearRegression

In [599]:
cols = ['year',
 'abi',
 'ticker',
 'company',
 'address_line_1',
 'city',
 'zipcode',
 'location_employee_size_code',
 'location_sales_volume_code',
 'primary_naics_code',
 'sic_code',
 'sic6_descriptions_sic',
 'business_status_code',
 'office_size_code',
 'company_holding_status',
 'parent_employee_size_code',
 'parent_sales_volume_code',
 'census_tract',
 'cbsa_code',
 'year_established',
 'employee_size_location',
 'sales_volume_location',
 'parent_actual_employee_size',
 'parent_actual_sales_volume',
 'latitude',
 'longitude']

bus = pd.read_csv("../data/chi_bus.csv", sep='\t', names=cols)

/Users/chrisolen/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (2,15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [600]:
bus.shape

(1144876, 26)

In [601]:
# filtering for useful features

bus = bus[['abi','primary_naics_code','company','year','business_status_code','company_holding_status','census_tract','year_established',
    'employee_size_location','sales_volume_location',
    'latitude','longitude']]

In [602]:
# understanding business status code

"""
Business Status Code:

1: Headquarter
2: Branch
3: Subsidiary
9: Single Location

"""

bus['business_status_code'].value_counts()

9    1034553
2     101871
3       4241
1       4211
Name: business_status_code, dtype: int64

In [603]:
# understanding business status holding

"""
Business Status Holding:

I'm gonna assume the 712 are publicly traded and the others aren't?

"""

bus['company_holding_status'].isna().value_counts()

True     1144164
False        712
Name: company_holding_status, dtype: int64

In [604]:
# understanding company holding status

bus['company_holding_status'].isna().value_counts()

True     1144164
False        712
Name: company_holding_status, dtype: int64

In [605]:
bus['primary_naics_code'].isna().value_counts()

False    1143647
True        1229
Name: primary_naics_code, dtype: int64

In [606]:
# remove naics code nulls and convert to string

bus = bus[~bus['primary_naics_code'].isna()]
bus['primary_naics_code'] = (bus['primary_naics_code'].astype(int)).astype(str)

In [607]:
# filtering for naics codes

"""

NAICS CODE 445110: Supermarkets and Other Grocery (except Convenience)
NAICS CODE 447110: Gasoline Stations with Convenience Stores
NAICS CODE 445120: Convenience Stores

"""

groc = bus[(bus['primary_naics_code'].str.contains('445110')) | (bus['primary_naics_code'].str.contains('447110')) | (bus['primary_naics_code'].str.contains('445120'))]


In [608]:
groc.shape

(14851, 12)

In [609]:
# understanding year distribution 

groc['year'].value_counts()

2013    1985
2014    1808
2012    1706
2015    1623
2009    1602
2010    1591
2011    1575
2016    1391
2017    1187
2018     383
Name: year, dtype: int64

In [610]:
# filtering for 2016 and 2017

groc_2016 = groc[groc['year']==2016]
groc_2017 = groc[groc['year']==2017]

In [611]:
demand = pd.merge(left=groc_2017, right=groc_2016, left_on='abi', right_on='abi')

In [612]:
demand.shape

(1112, 23)

In [613]:
demand.drop(['primary_naics_code_y','company_y','year_y','business_status_code_y',
            'company_holding_status_y','census_tract_y','year_established_y',
            'company_y','year_y','business_status_code_y',
            'latitude_y','longitude_y'], inplace=True, axis=1)

cols = [demand.columns[i].replace("_x","_2017").replace("_y","_2016") for i in range(len(demand.columns))]
replacing = {i:j for (i,j) in zip(demand.columns,cols)}
demand.rename(columns=replacing, inplace=True)

In [614]:
# connect to graph db

uri = "bolt://localhost:7687"
graph = Graph(uri, auth=("neo4j", "password"))



In [615]:
# pulling out coordinates for each store

coordinates = list(zip(demand["longitude_2017"],demand["latitude_2017"]))

In [616]:
with open('dicts/neighborhood_polys.json','r') as f:
    neighborhoods = json.load(f)

In [617]:
# laoding average property value for the neighborhood in which the stores is located

demand['neighborhood_avg_property_value'] = np.nan

outside_city = []

for i in range(len(coordinates)):

    point_district = utilities.point_lookup(neighborhoods,coordinates[i])
    
    try:
        result = float(dict(pd.DataFrame(graph.run('match (a:neighborhood) where a.name = "{}" return a'.format(point_district)). \
                            to_table()).iloc[0,0])['avg_property_value'])
    except:
        outside_city.append((i, coordinates[i]))
        pass
     
    ## coordinates and df indices should be the same ## 
    demand['neighborhood_avg_property_value'].iloc[i] = result
    
for i in range(len(outside_city)): # for the few coordinates that lie just outside the city
    
    point_district = utilities.closest_to(neighborhoods,outside_city[i][1])
    
    result = float(dict(pd.DataFrame(graph.run('match (a:neighborhood) where a.name = "{}" return a'.format(point_district)). \
                            to_table()).iloc[0,0])['avg_property_value'])
    
    
    demand['neighborhood_avg_property_value'].iloc[outside_city[i][0]] = result
    

/Users/chrisolen/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [618]:
# Loading number of property crimes per neighborhood into demand model for each store

demand['neighborhood_property_crimes'] = np.nan

outside_city = []

for i in range(len(coordinates)):

    point_district = utilities.point_lookup(neighborhoods,coordinates[i])
    
    try:
        result = float(dict(pd.DataFrame(graph.run('match (a:neighborhood) where a.name = "{}" return a'.format(point_district)). \
                            to_table()).iloc[0,0])['n_property_crimes'])
    except:
        outside_city.append((i, coordinates[i]))
        pass
     
    ## coordinates and df indices should be the same ## 
    demand['neighborhood_property_crimes'].iloc[i] = result
    
for i in range(len(outside_city)): # for the few coordinates that lie just outside the city
    
    point_district = utilities.closest_to(neighborhoods,outside_city[i][1])
    
    result = float(dict(pd.DataFrame(graph.run('match (a:neighborhood) where a.name = "{}" return a'.format(point_district)). \
                            to_table()).iloc[0,0])['n_property_crimes'])
    
    
    demand['neighborhood_property_crimes'].iloc[outside_city[i][0]] = result
    

In [619]:
# Loading average property values of neighborhoods surrounding the neighborhood of the given store

demand['surrounding_neighborhood_avg_property_value'] = np.nan

outside_city = []

for i in range(len(coordinates)):

    point_district = utilities.point_lookup(neighborhoods,coordinates[i])
    
    try:
        result = pd.DataFrame(graph.run('match (a:neighborhood)-[:NEXT_TO]->(b) where a.name = "{}" return b'.format(point_district)). \
                            to_table())

        n_next_door = len(result[0])

        neighboring_means = []

        for j in range(n_next_door):
            neighboring_mean = float(dict(result[0][j])['avg_property_value'])
            neighboring_means.append(neighboring_mean)
            
        surrounding_mean = np.nanmean(neighboring_means)
            
    except:
        outside_city.append((i, coordinates[i]))
     
    ## coordinates and df indices should be the same ## 
    demand['surrounding_neighborhood_avg_property_value'].iloc[i] = surrounding_mean
    
for i in range(len(outside_city)): # for the few coordinates that lie just outside the city
    
    point_district = utilities.closest_to(neighborhoods,outside_city[i][1])
    
    result = pd.DataFrame(graph.run('match (a:neighborhood)-[:NEXT_TO]->(b) where a.name = "{}" return b'.format(point_district)). \
                            to_table())

    n_next_door = len(result[0])

    neighboring_means = []

    for i in range(n_next_door):
        neighboring_mean = float(dict(result[0][i])['avg_property_value'])
        neighboring_means.append(neighboring_mean)
        
    surrounding_mean = np.nanmean(neighboring_means)
    
    demand['surrounding_neighborhood_avg_property_value'].iloc[outside_city[i][0]] = surrounding_mean
    

In [620]:
# Loading number of property crimes of neighborhoods surrounding the neighborhood of the given store

demand['surrounding_neighborhood_property_crimes'] = np.nan

outside_city = []

for i in range(len(coordinates)):

    point_district = utilities.point_lookup(neighborhoods,coordinates[i])

    try:
        result = pd.DataFrame(graph.run('match (a:neighborhood)-[:NEXT_TO]->(b) where a.name = "{}" return b'.format(point_district)). \
                            to_table())

        n_next_door = len(result[0])

        neighboring_means = []

        for j in range(n_next_door):
            neighboring_mean = float(dict(result[0][j])['n_property_crimes'])
            neighboring_means.append(neighboring_mean)
            
        surrounding_mean = np.nanmean(neighboring_means)
        
        ## coordinates and df indices should be the same ## 
        demand['surrounding_neighborhood_property_crimes'].iloc[i] = surrounding_mean
            
    except:
        outside_city.append((i, coordinates[i]))
        
    
for i in range(len(outside_city)): # for the few coordinates that lie just outside the city
    
    point_district = utilities.closest_to(neighborhoods,outside_city[i][1])
    
    result = pd.DataFrame(graph.run('match (a:neighborhood)-[:NEXT_TO]->(b) where a.name = "{}" return b'.format(point_district)). \
                            to_table())

    n_next_door = len(result[0])

    neighboring_means = []

    for i in range(n_next_door):
        neighboring_mean = float(dict(result[0][i])['n_property_crimes'])
        neighboring_means.append(neighboring_mean)
        
    surrounding_mean = np.nanmean(neighboring_means)
    
    demand['surrounding_neighborhood_property_crimes'].iloc[outside_city[i][0]] = surrounding_mean
    

In [621]:
demand = demand[(demand['sales_volume_location_2017'].notna()) &
               (demand['sales_volume_location_2016'].notna()) &
               (demand['neighborhood_property_crimes'].notna()) &
               (demand['neighborhood_avg_property_value'].notna()) &
               (demand['surrounding_neighborhood_avg_property_value'].notna()) &
               (demand['surrounding_neighborhood_property_crimes'].notna())]

In [622]:
X = np.array(demand[["sales_volume_location_2016","neighborhood_avg_property_value",
                     "neighborhood_property_crimes",
           "surrounding_neighborhood_avg_property_value","surrounding_neighborhood_property_crimes"]])



In [623]:
y = np.array(demand["sales_volume_location_2017"])

In [624]:
lr = LinearRegression()


In [625]:
model = lr.fit(X,y)

In [626]:
model.coef_

array([ 0.93421877, -0.01227086, -0.12626835, -0.01550418,  0.12616219])

In [627]:
model.intercept_

316.22113961802916

In [628]:
demand.to_csv("demand.csv")